<a href="https://colab.research.google.com/github/hemanthkumar17/Image-Processing-Lab/blob/main/IP_Lab_cheatsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Transforms

In [ ]:
def fft1d(m):
  fft1d = np.zeros([m, m], dtype=complex)
  for u in range(m):
    for x in range(m):
      fft1d[u][x] = np.round(np.exp(-1j*2*np.pi*x*u/m), 10)
  ifft1d = np.conj(fft1d)
  return fft1d, ifft1d

In [ ]:
def dct1d(N):
  dct_d = np.zeros([N, N])

  for k in range(N):
    for n in range(N):
      dct_d[k, n] = np.cos(np.pi*((n+0.5)/N * k))
  return dct_d

In [ ]:
def walsh1d(N):
  def getBinaries(n):
    binary = [int(x) for x in bin(n)[2:]]
    return [0]*(int(np.log2(N))-len(binary)) + binary
  G1d = np.zeros([N, N])
  for x in range(N):
    for u in range(N):
      bx = getBinaries(x)
      bu = getBinaries(u)
      G1d[x, u] = (-1) ** sum(np.multiply(bx, bu[::-1]))
  return G1d

In [ ]:
def hadamard1d(N):
  def getBinaries(n):
    binary = [int(x) for x in bin(n)[2:]]
    return [0]*(int(np.log2(N))-len(binary)) + binary
  H1d = np.zeros([N, N])
  for x in range(N):
    for u in range(N):
      bx = getBinaries(x)
      bu = getBinaries(u)
      H1d[x, u] = (-1) ** sum(np.multiply(bx, bu))
  return H1d

In [ ]:
def haar1DBasis(N):
  val = np.zeros((N**2, 2))
  for k in range(1, N**2):
    val[k][0] = np.floor(np.log2(k))
    val[k][1] = k - 2**val[k][0] + 1

  haar1d = np.zeros((N, N))
  haar1d[0] = [1] * N
  for k in range(1, N):
    for m in range(N):
      t = m/N
      p, q = val[k]
      p2 = 2 ** p
      print(k, t, p, q)
      print((q - 0.5)/ p2, q/ p2, (q - 1)/ p2)
      if t >= (q - 0.5)/ p2 and t < q/ p2:
        haar1d[k, m] = -np.sqrt(p2)
      elif t >= (q - 1)/ p2 and t < (q - 0.5)/ p2:
        haar1d[k, m] = np.sqrt(p2)
  return haar1d

### Convolution

In [ ]:
def convolve_2D(image, kernel):
    return ccorrelation_2d(image, np.flipud(np.fliplr(kernel)))

def ccorrelation_2d(image, kernel):
    paddingl = kernel.shape[0] - 1
    paddingu = kernel.shape[1] - 1
    # Shape of Output Convolution
    xOutput = int(((image.shape[0] - kernel.shape[0] + 2 * paddingl) ) + 1)
    yOutput = int(((image.shape[1] - kernel.shape[1] + 2 * paddingu) ) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    imagePadded = np.pad(image, ((paddingl, paddingl), (paddingu, paddingu)))
    print(imagePadded)

    # Iterate through image
    for y in range(yOutput):
        for x in range(xOutput):
            output[x, y] = np.sum(kernel * imagePadded[x: x + kernel.shape[0], y: y + kernel.shape[1]])
    return output

In [ ]:
convolve_2D(np.array([[1, 2, 1], [1, 2, 1], [3, 3, 3]]), np.array([[1, 1], [-1, -1]]))

[[0 0 0 0 0]
 [0 1 2 1 0]
 [0 1 2 1 0]
 [0 3 3 3 0]
 [0 0 0 0 0]]


array([[ 1.,  3.,  3.,  1.],
       [ 0.,  0.,  0.,  0.],
       [ 2.,  3.,  3.,  2.],
       [-3., -6., -6., -3.]])

### Filters

In [ ]:
robertsFilterx = np.array([[1, 0], [0, -1]])
robertsFiltery = np.flipud(robertsFilterx).transpose()

sobelFilterx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobelFiltery = np.flipud(sobelFilterx).transpose()

prewittFilterx = np.array([[-1, 0, 1]]*3)
prewittFiltery = np.flipud(prewittFilterx).transpose()
print(prewittFiltery)

[[-1 -1 -1]
 [ 0  0  0]
 [ 1  1  1]]


### Hist Equalization

In [ ]:
def histEqualization(img, max):
  hist, _ = np.histogram(img.flatten(), bins=range(np.max(img) + 2))
  pdf = hist / (img.shape[0] * img.shape[1])
  cdf = np.zeros(np.max(img) + 2)
  for i in range(np.max(img) + 1):
    cdf[i] = cdf[i-1] + pdf[i]
    cdf[i] = 1 if cdf[i] > 1 else cdf[i]
  cdf = cdf * max
  cdf = np.floor(cdf)
  new_image = np.zeros(img.shape)
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      new_image[i, j] = cdf[img[i, j]]
  return new_image, cdf

In [ ]:
img = np.array([[3, 2, 4, 5], [7, 7, 8, 2], [3, 1, 2, 3], [5, 4, 6, 7]])
print(histEqualization(img, 20))

(array([[ 8.,  5., 11., 13.],
       [18., 18., 20.,  5.],
       [ 8.,  1.,  5.,  8.],
       [13., 11., 15., 18.]]), array([ 0.,  1.,  5.,  8., 11., 13., 15., 18., 20.,  0.]))


### Hist Specialization

In [ ]:
def histSpecialisation(origImage, targetImage):
  equal1 = histEqualization(origImage)[1]
  equal2 = histEqualization(targetImage)[1]
  return list(map(lambda eq1val: np.where(equal2 >= eq1val)[0][0], equal1))

### Image clipping

In [ ]:
def imgClip(img, thresh):
  img[img<thresh] = 0
  return img
def imgWindow(img, window):
  img[img>window] = 0
  return img

### 4 Filters - avg, med, min, max

In [ ]:
def avgFilter(img):
  mask = np.ones([3, 3], dtype=int)/9
  avg_image = np.zeros(img.shape)
  for i in range(1, img.shape[0]-1):
    for j in range(1, img.shape[1]-1):
      for p in range(-1, 2):
        for q in range(-1, 2):
          avg_image[i, j] = avg_image[i, j] + img[i+p, j+q] * mask[p, q]
  return avg_image

In [ ]:
def medFilter(img):
  med_image = np.zeros(img.shape)
  for i in range(1, img.shape[0]-1):
    for j in range(1, img.shape[1]-1):
      temp = []
      for p in range(-1, 2):
        for q in range(-1, 2):
          temp = temp + [img[i+p, j+q]]
      med_image[i, j] = sorted(temp)[4]
  return med_image

In [ ]:
def minFilter(img):
  min_image = np.zeros(img.shape)
  for i in range(1, img.shape[0]-1):
    for j in range(1, img.shape[1]-1):
      temp = []
      for p in range(-1, 2):
        for q in range(-1, 2):
          temp = temp + [img[i+p, j+q]]
      min_image[i, j] = min(temp)
  return min_image

In [ ]:
def maxFilter(img):
  max_image = np.zeros(img.shape)
  for i in range(1, img.shape[0]-1):
    for j in range(1, img.shape[1]-1):
      temp = []
      for p in range(-1, 2):
        for q in range(-1, 2):
          temp = temp + [img[i+p, j+q]]
      max_image[i, j] = max(temp)
  return max_image

### Automatic Thresholding

In [ ]:
def autoThresh(img):
  T = img.mean()
  Tprev = 500
  while abs(T - Tprev) >= 1:
    A1 = img[img > T].mean()
    A2 = img[img <= T].mean()
    Tprev = T
    T = (A1 + A2) / 2
  print(T)
  return T

### OTSU Thresh

In [ ]:
def otsuThresh(img):
  hist, _ = np.histogram(img.flatten(), bins=range(257))
  pdf = hist / (img.shape[0] * img.shape[1])

  cdf = np.zeros(256)
  wcdf = np.zeros(256)
  w2cdf = np.zeros(256)

  cdf[0] = pdf[0]
  for i in range(1, 256):
    cdf[i] = cdf[i-1] + pdf[i]
    wcdf[i] = wcdf[i-1] + i * pdf[i]
    w2cdf[i] = w2cdf[i-1] + i * i * pdf[i]

  meang = wcdf[255]
  max = -1
  k_min = 0
  invar_min = 10000
  for k in range(256):

    # p0 = cdf[k]
    # p1 = 1 - p0
    p = np.array([cdf[k], 1 - cdf[k]])
    
    # w0 = wcdf[k]
    # w1 = wcdf[255] - w0
    w = np.array([wcdf[k], wcdf[255] - wcdf[k]])

    # w20 = w2cdf[k]
    # w21 = w2cdf[255] - w20
    w2 = np.array([w2cdf[k], w2cdf[255] - w2cdf[k]])

    if not min(p):
      continue

    # mean0 = w0 / p0
    # mean1 = w1 / p1
    mean = w / p
    
    # var0 = w20 / p0 - mean0 ** 2
    # var1 = w21 / p1 - mean1 ** 2
    var = w2/p - mean ** 2

    # invar = p0 * var0 + p1 * var1
    invar = np.sum(p * var)

    if invar < invar_min:
      invar_min = invar
      k_min = k

  print(icvar_min)
  print(k_min)

  img[img < k_min] = 0
  img[img != 0] = 255
  # print(k_min)
  return img

### Bitplane Slicing

In [ ]:
def bitPlaneSlicing(img):
  bitSlicedImage = np.zeros([img.shape[0], img.shape[1], 9])
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      binary = (np.binary_repr(img[i][j] ,width=8)) # width = no. of bits
      for bit in range(8):
        bitSlicedImage[i, j, 8-bit] = int(binary[bit]) * (2 ** (7-bit))
  print(bitSlicedImage.shape)
  fig = plt.figure(figsize=(20, 10))
  for i in range(8):
    plt.subplot(2, 4, i+1)
    plt.imshow(bitSlicedImage[:, :, 8-i], cmap="gray", vmax=255, vmin=0)

### Huffman Coding

In [ ]:
class Node:
  def __init__(self, frequency, char = None, left=None, right=None):
    self.left = left
    self.right = right
    self.frequency = frequency
    self.char = char
    self.direction = ""

  def printNode(self, val=""):
    newVal = val + str(self.direction)
    if self.left:
      self.left.printNode(newVal)
    if self.right:
      self.right.printNode(newVal)
    if not self.left and not self.right:
      print(f"{self.char}: {newVal}")


def HuffmanCoding(code):
  frequency = dict()

  for char in code:
    if char not in frequency.keys():
      frequency[char] =  0
    frequency[char] += 1

  queue = [Node(freq, char) for char, freq in sorted(frequency.items(), key=lambda item: item[1])]

  while len(queue) > 1:
    left = queue.pop(0)
    right = queue.pop(0)

    left.direction = "0"
    right.direction = "1"
    queue.append(Node(left.frequency+right.frequency, left.char+right.char, left, right))

    queue = sorted(queue, key= lambda x: x.frequency)
  root = queue[0]
  encodings = {x:y for x, y in queue[0].printNode()}
  print(code)
  encoded = ""
  for char in code:
    encoded += encodings[char]
  return encoded

### Arithmetic Encoding

In [ ]:
def seqCode(seqstrip, lb, ub, prob, code):
  if not seqstrip:
    return (round(lb, 7), round(ub, 7))
  print("Sequencestrip\n" + seqstrip + "\nWith lower bound and upper bound")
  print(round(lb, 7), round(ub))
  curr = seqstrip[0]
  seqstrip = seqstrip[1:]
  newrange = ub - lb
  cprob = prob.cumsum()
  if not code[curr]:
    new_lb = lb
  else:
    new_lb = lb + newrange * cprob[code[curr] - 1]
  new_ub = lb + newrange * cprob[code[curr]]
  return seqCode(seqstrip, new_lb, new_ub, prob, code)

def arithEncoding(seq, codelen = 5):
  code = {}
  i = 0
  for c in seq:
    if c not in code:
      code[c] = i
      i += 1
  freq = np.zeros(len(set(seq)))
  for x in seq:
    freq[code[x]] += 1
  prob = freq / len(seq)

  encoding = []
  for i in range(len(seq) // codelen):
    e = seqCode(seq[i*codelen: (i+1)*codelen], 0, 1, prob, code)
    print("\nEncoding of strip " + seq[i*codelen: (i+1)*codelen] + ": ")
    print(e)
    print()
    encoding.append(e)
  for e in encoding:
    print(e)
  return encoding

### Region Growing

In [ ]:
class Point(object):
 def __init__(self,x,y):
  self.x = x
  self.y = y

def regionGrow(img, seed, thresh,p = 1):
 height, weight = img.shape
 visited = np.zeros(img.shape)
 queue = []
 queue.append(seed)
 growthpixels = [Point(-1, -1), Point(0, -1), Point(1, -1), Point(1, 0), Point(1, 1), Point(0, 1), Point(-1, 1), Point(-1, 0)]
 
 while queue:
  currentPoint = queue.pop(0)
  visited[currentPoint.x,currentPoint.y] = 1
  for i in range(8):
   tmpX = currentPoint.x + growthpixels[i].x
   tmpY = currentPoint.y + growthpixels[i].y
   if tmpX < 0 or tmpY < 0 or tmpX >= height or tmpY >= weight:
    continue
   if abs(int(img[currentPoint.x,currentPoint.y]) - int(img[tmpX,tmpY])) < thresh and visited[tmpX,tmpY] == 0:
    visited[tmpX,tmpY] = 1
    queue.append(Point(tmpX,tmpY))
 return visited

### Hough transform

In [ ]:
def houghTransform(img, a_min=0, a_max=2, b_min=None, b_max=None):
  a_min = 0
  a_max = 2
  x_max, y_max = img.shape[0]-1, img.shape[1]-1
  if not b_max:
    b_max = y_max - np.min(0, a_min * x_max)
  if not b_min:
    b_min = 0 - np.max(0, a_min * x_max)

  acc = np.zeros([a_max - a_min, b_max - b_min + 1])

  for x in range(x_max + 1):
    for y in range(y_max + 1):
      if img[x, y] != 0:
        for a in range(a_min, a_max):
          b = y - a * x
          acc[a, b] += 1
  return acc